In [2]:
!pip install qiskit
!pip install qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 6.0 MB/s eta 0:00:00


In [8]:
from qiskit import QuantumCircuit
from qiskit_aer import Aer
import numpy as np

def create_bell_pair():
    qc = QuantumCircuit(2, 2)
    qc.h(0)  # Hadamard on qubit 0
    qc.cx(0, 1)  # CNOT with control 0, target 1
    return qc

def measure_in_basis(qc, theta_A, theta_B):
    qc.ry(theta_A, 0)  # Rotate measurement basis for Alice
    qc.ry(theta_B, 1)  # Rotate measurement basis for Bob
    qc.measure([0, 1], [0, 1])
    return qc

def run_experiment(theta_A, theta_B, shots=1000):
    qc = create_bell_pair()
    qc = measure_in_basis(qc, theta_A, theta_B)
    simulator = Aer.get_backend('qasm_simulator')
    job = simulator.run(qc, shots=shots)  # Use backend.run() instead of execute
    result = job.result()
    counts = result.get_counts()
    return counts

def correlation(counts, shots):
    parity = counts.get('00', 0) + counts.get('11', 0) - counts.get('01', 0) - counts.get('10', 0)
    return parity / shots

def test_bell_inequality():
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    shots = 1000

    E = {}
    for i in range(4):
        for j in range(4):
            counts = run_experiment(angles[i], angles[j], shots)
            E[(i, j)] = correlation(counts, shots)

    S = E[(0, 1)] - E[(0, 3)] + E[(2, 1)] + E[(2, 3)]

    print(f"Bell's S value: {S}")
    if abs(S) > 2:
        print("Bell's inequality violated! Quantum entanglement confirmed.")
    else:
        print("No violation. Classical physics holds.")

test_bell_inequality()

Bell's S value: 2.816
Bell's inequality violated! Quantum entanglement confirmed.
